In [1]:
%matplotlib inline
import numpy as np
import nbimporter
from _helper_functions import find_nearest_sorted, find_nearest_sorted

Importing Jupyter notebook from _helper_functions.ipynb


In [2]:
#this is as optimized as possible using numpy slicing so calling this function SHOULD be very fast

#@jit
def mbes(Y, t, eta, kappa, deltac, gs, gperp, spins, gpar):
    ret = np.empty(len(Y), dtype=complex)
    ret[0] = -kappa*Y[0]-1j*deltac*Y[0]+np.sum(gs*Y[1::2])+eta
    ret[1::2] = -(gperp+gpar/2 + 1j*(spins*2*np.pi+deltac))*Y[1::2]+gs*Y[2::2]*Y[0]
    ret[2::2] = -gpar*(1+Y[2::2])-2*gs*(Y[1::2]*np.conj(Y[0])+np.conj(Y[1::2])*Y[0])
    return ret

#%%timeit gives
#10000 loops, best of 3: 45 µs per loop
#for 800 spins it becomes ~70µs. This is probably not the bottleneck


#this function is a lot slower, so be careful when using it
#@jit
def mbes_soc(Y, t, ilist, qlist, tlist, kappa, deltac, gs, gperp, spins, gpar):
    ret = np.empty(len(Y), dtype=complex)
    idx = find_nearest_sorted(tlist, t)
    ret[0] = -kappa*Y[0]-1j*deltac*Y[0]+np.sum(gs*Y[1::2])+ilist[idx]-1j*qlist[idx]
    ret[1::2] = -(gperp+gpar/2 +1j*(spins*2*np.pi+deltac))*Y[1::2]+gs*Y[2::2]*Y[0]
    ret[2::2] = -gpar*(1+Y[2::2])-2*gs*(Y[1::2]*np.conj(Y[0])+np.conj(Y[1::2])*Y[0])
    return ret

def mbes_soc_func(Y, t, func, func_args, kappa, deltac, gs, gperp, spins, gpar):
    pi, pq = func(t, *func_args)
    ret = np.empty(len(Y), dtype=complex)
    ret[0] = -kappa*Y[0]-1j*deltac*Y[0]+np.sum(gs*Y[1::2])+pi-1j*pq
    ret[1::2] = -(gperp+gpar/2 +1j*(spins*2*np.pi+deltac))*Y[1::2]+gs*Y[2::2]*Y[0]
    ret[2::2] = -gpar*(1+Y[2::2])-2*gs*(Y[1::2]*np.conj(Y[0])+np.conj(Y[1::2])*Y[0])
    return ret

def mbes_simulate_losses(Y, t, eta, kappain, kappa, deltac, gs, gperp, spins, gpar):
    ret = np.empty(len(Y), dtype=complex)
    ret[0] = -kappa*Y[0]-1j*deltac*Y[0]+np.sum(gs*Y[1::2])+np.sqrt(2*kappain)*eta
    ret[1::2] = -(gperp+gpar/2 + 1j*(spins*2*np.pi+deltac))*Y[1::2]+gs*Y[2::2]*Y[0]
    ret[2::2] = -gpar*(1+Y[2::2])-2*gs*(Y[1::2]*np.conj(Y[0])+np.conj(Y[1::2])*Y[0])
    return ret
